In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import missingno as msno

import dask.dataframe as dd
import dask.array as da

plt.rcParams['figure.dpi'] = 300
plt.rc('font', family='NanumBarunGothic') 

In [2]:
import matplotlib
import matplotlib.font_manager

[f.name for f in matplotlib.font_manager.fontManager.ttflist if 'Nanum' in f.name]


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import numpy as np

data = np.random.randint(-100, 100, 50).cumsum()
data

In [5]:
plt.rcParams["font.family"] = 'NanumGothicCoding'

In [6]:
!yes|sudo apt-get install fonts-nanum*

In [7]:
!sudo fc-cache -fv

In [8]:
ls -l /usr/share/fonts/truetype/

In [9]:
ls -l /usr/share/fonts/truetype/nanum/

In [10]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
fontprop = fm.FontProperties(fname=path, size=18)

plt.plot(range(50), data, 'r')
plt.title('가격변동 추이', fontproperties=fontprop)
plt.ylabel('가격', fontproperties=fontprop)
plt.show()

# 데이터 불러 오기

In [11]:
df_2022_Q1 = dd.read_csv('../data/dong_Q_guplus/D/df_2023_Q1_M_E_N2.csv', encoding = 'utf-8')

비정기적 이동 시간때 설명  
심야 : 0 ~ 5시  
오전 : 6 ~ 10시  
점심 : 11 ~ 12시  
오후 : 13 ~ 16시  
저녁 : 17 ~ 18시  
야간 : 19 ~ 23시


In [12]:
df_2022_Q1.head()

In [13]:
df_2022_Q1.head().info()

# 기타 이동 기준 : 시군구 / 행정동별 상위 10개 지역

In [14]:
df_2022_Q1_t10 = df_2022_Q1.groupby('도착 시군구 코드')['이동인구(합)'].sum().compute().nlargest(10)
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
sns.barplot(x=df_2022_Q1_t10.index, y=df_2022_Q1_t10.values, color='orange')
plt.title("상위 10개 도착 시군구 코드별 이동인구(합) 합계")
plt.xlabel("도착 시군구 코드")
plt.ylabel("이동인구(합) 합계(단위:천만)")
plt.xticks(rotation=45)

# 원래의 데이터프레임 df_2022_Q1을 사용하여 도착 행정동 별로 그룹화
df_2022_Q1_dong = df_2022_Q1.groupby('도착 행정동 코드')['이동인구(합)'].sum().compute().nlargest(10)
plt.subplot(1,2,2)
sns.barplot(x=df_2022_Q1_dong.index, y=df_2022_Q1_dong.values, color='orange')
plt.title("상위 10개 도착 행정동 코드별 이동인구(합) 합계")
plt.xlabel("도착 행정동 코드")
plt.ylabel("이동인구(핕) 핍곗 (단위:백만)")
plt.xticks(rotation=45)

# 레이아웃 조절 및 차트 출력
plt.tight_layout()
plt.show()


강남 서초 송파 마포 중로 중구  영등포 // 성수, 이태원

In [15]:
df_2022_Q1_t10.head()

In [16]:
df_2022_Q1.head().info()

# 기타 이동 기준 : 특정 요일에 가장 많이 방문하는 시군구 상위 10개 지역

In [17]:
# 요일 리스트 생성
days = ['월', '화', '수', '목', '금', '토', '일']

# 요일별로 상위 10개 도착지와 이동인구 합계 계산
top_g_by_day = df_2022_Q1.groupby(['요일', '도착 시군구 코드'])['이동인구(합)'].sum().compute().groupby('요일').nlargest(10)

for day in days:
    # 특정 요일에 대한 데이터 추출
    df_day = top_g_by_day[day].reset_index(level=0, drop=True)
    
    # 막대 그래프
    plt.figure(figsize=(15, 10))
    plt.subplot(2,2,1)
    sns.barplot(x=df_day.index, y=df_day.values)
    plt.title(f"{day}요일에 가장 많이 방문하는 시군구 상위 10곳")
    
    # 원형 그래프
    plt.subplot(2,2,2)
    plt.pie(df_day.values,
            labels=df_day.index,
            autopct='%1.1f%%')
    
   # 꺽은선 그래프 (선그래프는 순서가 중요하기 때문에 정렬된 값으로 생성)
    df_sorted = df_day.sort_index()
    plt.subplot(2,2,3)
    sns.lineplot(x=df_sorted.index,y=df_sorted.values)

   # 레이아웃 조절 및 차트 출력
    plt.tight_layout()
    plt.show()

# 시군구 기준 심야 시간  
## 1) 이동인구가 가장 많은 상위 10개  
## 2) 성별 분포
## 3) 요일 분포
## 4) 나이 기준으로 성별 분포
### 심야 기준 ( 0 ~ 6시 사이)

In [18]:
    filtered_df_AE = df_2022_Q1[(df_2022_Q1['이동시간'] == '심야')]

    filtered_df_AE = filtered_df_AE.compute()

    sorted_df = filtered_df_AE.groupby('도착 시군구 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    plt.figure(figsize=(10,6))
    sorted_df.plot(kind='bar', color='skyblue')
    plt.title('심야 0 ~ 6시 전 도착 기준 시군구 코드별 이동인구 합 상위 10개')
    plt.xlabel('도착 시군구 코드')
    plt.ylabel('이동인구 합계')
    plt.show()

    요일순서 = ['월', '화', '수', '목', '금', '토', '일']

    # 도착 시군구 코드와 성별 나이 요일을 그룹화하고, 각 그룹의 이동인구(합)을 합산
    grouped_df = filtered_df_AE.groupby(['도착 시군구 코드', '성별'])['이동인구(합)'].sum().reset_index()
    grouped_df_2 = filtered_df_AE.groupby(['도착 시군구 코드', '나이'])['이동인구(합)'].sum().reset_index()
    grouped_df_3 = filtered_df_AE.groupby(['도착 시군구 코드', '요일'])['이동인구(합)'].sum().reset_index()
    grouped_df_age_gender = filtered_df_AE.groupby(['도착 시군구 코드', '나이', '성별'])['이동인구(합)'].sum().unstack().reset_index()


    # 상위 10개 도착 시군 구 코드만 선택
    top_regions = sorted_df.index.tolist()
    # grouped_df = grouped_df[grouped_df['도착 시군구 코드'].isin(top_regions)]
    # grouped_df_2 = grouped_df_2[grouped_df_2['도착 시군구 코드'].isin(top_regions)]
    grouped_df_3 = grouped_df_3[grouped_df_3['도착 시군구 코드'].isin(top_regions)]

    for region in top_regions:
        df_region = grouped_df[grouped_df['도착 시군구 코드'] == region]
        df_region_2 = grouped_df_2[grouped_df_2['도착 시군구 코드'] == region]
        df_region_3 = grouped_df_3[grouped_df_3['도착 시군구 코드'] == region]
        
        # 요일순서에 따라 데이터 정렬
        df_region_3['요일'] = pd.Categorical(df_region_3['요일'], categories=요일순서, ordered=True)
        df_region_3.sort_values(by='요일', inplace=True)
        
        # 서브 플롯 설정 (1x3)
        plt.figure(figsize=(18,6))

        # 원형그래프
        plt.subplot(1, 3, 1)
        plt.pie(df_region['이동인구(합)'], labels=df_region['성별'], autopct='%1.1f%%')
        plt.title(f'{region} - 성별 분포 ')
        
        # # 꺽은선그래프
        # plt.subplot(1, 3, 2)
        # df_region_2.set_index('나이')['이동인구(합)'].plot(kind='line', marker='o')
        # plt.title(f'{region} - 나이 ')
        
        # 막대그래프
        plt.subplot(1 , 3 , 2)
        df_region_3.set_index('요일')['이동인구(합)'].plot(kind='bar', color='skyblue')
        plt.title(f'{region} - 요일 ')

        # 꺽은선 그래프 & 막대그래프로 나이-성별재분류 추가 
        plt.subplot(1, 3, 2)
        df_region_age_gender = grouped_df_age_gender[grouped_df_age_gender['도착 시군구 코드'] == region].set_index('나이')
        df_region_age_gender.plot(kind='bar', stacked=True, figsize=(6, 4))  # 사이즈 조정
        df_region_age_gender.plot(figsize=(6, 4))   # 사이즈 조정
        plt.title(f'Age and Gender Distribution in {region}')
        plt.xlabel('Age')
        plt.ylabel('Count')
        plt.grid()
        
        # 그래프 출력
        plt.show()

# 시군구 기준 오전 시간  
## 1) 이동인구가 가장 많은 상위 10개  
## 2) 성별 분포
## 3) 요일 분포
## 4) 나이 기준으로 성별 분포 
### 오전 기준( 6 ~ 11시 사이)

In [19]:
    filtered_df_BE = df_2022_Q1[(df_2022_Q1['이동시간'] == '오전')]

    filtered_df_BE = filtered_df_BE.compute()

    sorted_df = filtered_df_BE.groupby('도착 시군구 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    plt.figure(figsize=(10,6))
    sorted_df.plot(kind='bar', color='skyblue')
    plt.title('오전 6 ~ 11시 전 도착 기준 시군구 코드별 이동인구 합 상위 10개')
    plt.xlabel('도착 시군구 코드')
    plt.ylabel('이동인구 합계')
    plt.show()

    요일순서 = ['월', '화', '수', '목', '금', '토', '일']

    # 도착 시군구 코드와 성별 나이 요일을 그룹화하고, 각 그룹의 이동인구(합)을 합산
    grouped_df = filtered_df_BE.groupby(['도착 시군구 코드', '성별'])['이동인구(합)'].sum().reset_index()
    grouped_df_2 = filtered_df_BE.groupby(['도착 시군구 코드', '나이'])['이동인구(합)'].sum().reset_index()
    grouped_df_3 = filtered_df_BE.groupby(['도착 시군구 코드', '요일'])['이동인구(합)'].sum().reset_index()
    grouped_df_age_gender = filtered_df_BE.groupby(['도착 시군구 코드', '나이', '성별'])['이동인구(합)'].sum().unstack().reset_index()


    # 상위 10개 도착 시군 구 코드만 선택
    top_regions = sorted_df.index.tolist()
    # grouped_df = grouped_df[grouped_df['도착 시군구 코드'].isin(top_regions)]
    # grouped_df_2 = grouped_df_2[grouped_df_2['도착 시군구 코드'].isin(top_regions)]
    grouped_df_3 = grouped_df_3[grouped_df_3['도착 시군구 코드'].isin(top_regions)]

    for region in top_regions:
        df_region = grouped_df[grouped_df['도착 시군구 코드'] == region]
        df_region_2 = grouped_df_2[grouped_df_2['도착 시군구 코드'] == region]
        df_region_3 = grouped_df_3[grouped_df_3['도착 시군구 코드'] == region]
        
        # 요일순서에 따라 데이터 정렬
        df_region_3['요일'] = pd.Categorical(df_region_3['요일'], categories=요일순서, ordered=True)
        df_region_3.sort_values(by='요일', inplace=True)
        
        # 서브 플롯 설정 (1x3)
        plt.figure(figsize=(18,6))

        # 원형그래프
        plt.subplot(1, 3, 1)
        plt.pie(df_region['이동인구(합)'], labels=df_region['성별'], autopct='%1.1f%%')
        plt.title(f'{region} - 성별 분포 ')
        
        # # 꺽은선그래프
        # plt.subplot(1, 3, 2)
        # df_region_2.set_index('나이')['이동인구(합)'].plot(kind='line', marker='o')
        # plt.title(f'{region} - 나이 ')
        
        # 막대그래프
        plt.subplot(1 , 3 , 2)
        df_region_3.set_index('요일')['이동인구(합)'].plot(kind='bar', color='skyblue')
        plt.title(f'{region} - 요일 ')

        # 꺽은선 그래프 & 막대그래프로 나이-성별재분류 추가 
        plt.subplot(1, 3, 2)
        df_region_age_gender = grouped_df_age_gender[grouped_df_age_gender['도착 시군구 코드'] == region].set_index('나이')
        df_region_age_gender.plot(kind='bar', stacked=True, figsize=(6, 4))  # 사이즈 조정
        df_region_age_gender.plot(figsize=(6, 4))   # 사이즈 조정
        plt.title(f'Age and Gender Distribution in {region}')
        plt.xlabel('Age')
        plt.ylabel('Count')
        plt.grid()
        
        # 그래프 출력
        plt.show()

# 시군구 기준 점심 시간  
## 1) 이동인구가 가장 많은 상위 10개  
## 2) 성별 분포
## 3) 요일 분포
## 4) 나이 기준으로 성별 분포
### 점심 기준 ( 11 ~ 13시 사이)

In [20]:
    filtered_df_CE = df_2022_Q1[(df_2022_Q1['이동시간'] == '점심')]

    filtered_df_CE = filtered_df_CE.compute()

    sorted_df = filtered_df_CE.groupby('도착 시군구 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    plt.figure(figsize=(10,6))
    sorted_df.plot(kind='bar', color='skyblue')
    plt.title('점심 11 ~ 13시 전 도착 기준 시군구 코드별 이동인구 합 상위 10개')
    plt.xlabel('도착 시군구 코드')
    plt.ylabel('이동인구 합계')
    plt.show()

    요일순서 = ['월', '화', '수', '목', '금', '토', '일']

    # 도착 시군구 코드와 성별 나이 요일을 그룹화하고, 각 그룹의 이동인구(합)을 합산
    grouped_df = filtered_df_CE.groupby(['도착 시군구 코드', '성별'])['이동인구(합)'].sum().reset_index()
    grouped_df_2 = filtered_df_CE.groupby(['도착 시군구 코드', '나이'])['이동인구(합)'].sum().reset_index()
    grouped_df_3 = filtered_df_CE.groupby(['도착 시군구 코드', '요일'])['이동인구(합)'].sum().reset_index()
    grouped_df_age_gender = filtered_df_CE.groupby(['도착 시군구 코드', '나이', '성별'])['이동인구(합)'].sum().unstack().reset_index()


    # 상위 10개 도착 시군 구 코드만 선택
    top_regions = sorted_df.index.tolist()
    # grouped_df = grouped_df[grouped_df['도착 시군구 코드'].isin(top_regions)]
    # grouped_df_2 = grouped_df_2[grouped_df_2['도착 시군구 코드'].isin(top_regions)]
    grouped_df_3 = grouped_df_3[grouped_df_3['도착 시군구 코드'].isin(top_regions)]

    for region in top_regions:
        df_region = grouped_df[grouped_df['도착 시군구 코드'] == region]
        df_region_2 = grouped_df_2[grouped_df_2['도착 시군구 코드'] == region]
        df_region_3 = grouped_df_3[grouped_df_3['도착 시군구 코드'] == region]
        
        # 요일순서에 따라 데이터 정렬
        df_region_3['요일'] = pd.Categorical(df_region_3['요일'], categories=요일순서, ordered=True)
        df_region_3.sort_values(by='요일', inplace=True)
        
        # 서브 플롯 설정 (1x3)
        plt.figure(figsize=(18,6))

        # 원형그래프
        plt.subplot(1, 3, 1)
        plt.pie(df_region['이동인구(합)'], labels=df_region['성별'], autopct='%1.1f%%')
        plt.title(f'{region} - 성별 분포 ')
        
        # # 꺽은선그래프
        # plt.subplot(1, 3, 2)
        # df_region_2.set_index('나이')['이동인구(합)'].plot(kind='line', marker='o')
        # plt.title(f'{region} - 나이 ')
        
        # 막대그래프
        plt.subplot(1 , 3 , 2)
        df_region_3.set_index('요일')['이동인구(합)'].plot(kind='bar', color='skyblue')
        plt.title(f'{region} - 요일 ')

        # 꺽은선 그래프 & 막대그래프로 나이-성별재분류 추가 
        plt.subplot(1, 3, 2)
        df_region_age_gender = grouped_df_age_gender[grouped_df_age_gender['도착 시군구 코드'] == region].set_index('나이')
        df_region_age_gender.plot(kind='bar', stacked=True, figsize=(6, 4))  # 사이즈 조정
        df_region_age_gender.plot(figsize=(6, 4))   # 사이즈 조정
        plt.title(f'Age and Gender Distribution in {region}')
        plt.xlabel('Age')
        plt.ylabel('Count')
        plt.grid()
        
        # 그래프 출력
        plt.show()

# 시군구 기준 오후 시간  
## 1) 이동인구가 가장 많은 상위 10개  
## 2) 성별 분포
## 3) 요일 분포
## 4) 나이 기준으로 성별 분포 
### 오후 기준 ( 13 ~ 17시 사이)

In [21]:
    filtered_df_DE = df_2022_Q1[(df_2022_Q1['이동시간'] == '오후')]

    filtered_df_DE = filtered_df_DE.compute()

    sorted_df = filtered_df_DE.groupby('도착 시군구 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    plt.figure(figsize=(10,6))
    sorted_df.plot(kind='bar', color='skyblue')
    plt.title('오후 13 ~ 16 시 전 도착 기준 시군구 코드별 이동인구 합 상위 10개')
    plt.xlabel('도착 시군구 코드')
    plt.ylabel('이동인구 합계')
    plt.show()

    요일순서 = ['월', '화', '수', '목', '금', '토', '일']

    # 도착 시군구 코드와 성별 나이 요일을 그룹화하고, 각 그룹의 이동인구(합)을 합산
    grouped_df = filtered_df_DE.groupby(['도착 시군구 코드', '성별'])['이동인구(합)'].sum().reset_index()
    grouped_df_2 = filtered_df_DE.groupby(['도착 시군구 코드', '나이'])['이동인구(합)'].sum().reset_index()
    grouped_df_3 = filtered_df_DE.groupby(['도착 시군구 코드', '요일'])['이동인구(합)'].sum().reset_index()
    grouped_df_age_gender = filtered_df_DE.groupby(['도착 시군구 코드', '나이', '성별'])['이동인구(합)'].sum().unstack().reset_index()


    # 상위 10개 도착 시군 구 코드만 선택
    top_regions = sorted_df.index.tolist()
    # grouped_df = grouped_df[grouped_df['도착 시군구 코드'].isin(top_regions)]
    # grouped_df_2 = grouped_df_2[grouped_df_2['도착 시군구 코드'].isin(top_regions)]
    grouped_df_3 = grouped_df_3[grouped_df_3['도착 시군구 코드'].isin(top_regions)]

    for region in top_regions:
        df_region = grouped_df[grouped_df['도착 시군구 코드'] == region]
        df_region_2 = grouped_df_2[grouped_df_2['도착 시군구 코드'] == region]
        df_region_3 = grouped_df_3[grouped_df_3['도착 시군구 코드'] == region]
        
        # 요일순서에 따라 데이터 정렬
        df_region_3['요일'] = pd.Categorical(df_region_3['요일'], categories=요일순서, ordered=True)
        df_region_3.sort_values(by='요일', inplace=True)
        
        # 서브 플롯 설정 (1x3)
        plt.figure(figsize=(18,6))

        # 원형그래프
        plt.subplot(1, 3, 1)
        plt.pie(df_region['이동인구(합)'], labels=df_region['성별'], autopct='%1.1f%%')
        plt.title(f'{region} - 성별 분포 ')
        
        # # 꺽은선그래프
        # plt.subplot(1, 3, 2)
        # df_region_2.set_index('나이')['이동인구(합)'].plot(kind='line', marker='o')
        # plt.title(f'{region} - 나이 ')
        
        # 막대그래프
        plt.subplot(1 , 3 , 2)
        df_region_3.set_index('요일')['이동인구(합)'].plot(kind='bar', color='skyblue')
        plt.title(f'{region} - 요일 ')

        # 꺽은선 그래프 & 막대그래프로 나이-성별재분류 추가 
        plt.subplot(1, 3, 2)
        df_region_age_gender = grouped_df_age_gender[grouped_df_age_gender['도착 시군구 코드'] == region].set_index('나이')
        df_region_age_gender.plot(kind='bar', stacked=True, figsize=(6, 4))  # 사이즈 조정
        df_region_age_gender.plot(figsize=(6, 4))   # 사이즈 조정
        plt.title(f'Age and Gender Distribution in {region}')
        plt.xlabel('Age')
        plt.ylabel('Count')
        plt.grid()
        
        # 그래프 출력
        plt.show()

# 시군구 기준 저녁 시간  
## 1) 이동인구가 가장 많은 상위 10개  
## 2) 성별 분포
## 3) 요일 분포
## 4) 나이 기준으로 성별 분포 
### 저녁 기준 ( 17 ~ 19시 사이)

In [22]:
    filtered_df_EE = df_2022_Q1[(df_2022_Q1['이동시간'] == '저녁')]

    filtered_df_EE = filtered_df_EE.compute()

    sorted_df = filtered_df_EE.groupby('도착 시군구 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    plt.figure(figsize=(10,6))
    sorted_df.plot(kind='bar', color='skyblue')
    plt.title('저녁 17 ~ 18시 전 도착 기준 시군구 코드별 이동인구 합 상위 10개')
    plt.xlabel('도착 시군구 코드')
    plt.ylabel('이동인구 합계')
    plt.show()

    요일순서 = ['월', '화', '수', '목', '금', '토', '일']

    # 도착 시군구 코드와 성별 나이 요일을 그룹화하고, 각 그룹의 이동인구(합)을 합산
    grouped_df = filtered_df_EE.groupby(['도착 시군구 코드', '성별'])['이동인구(합)'].sum().reset_index()
    grouped_df_2 = filtered_df_EE.groupby(['도착 시군구 코드', '나이'])['이동인구(합)'].sum().reset_index()
    grouped_df_3 = filtered_df_EE.groupby(['도착 시군구 코드', '요일'])['이동인구(합)'].sum().reset_index()
    grouped_df_age_gender = filtered_df_EE.groupby(['도착 시군구 코드', '나이', '성별'])['이동인구(합)'].sum().unstack().reset_index()


    # 상위 10개 도착 시군 구 코드만 선택
    top_regions = sorted_df.index.tolist()
    # grouped_df = grouped_df[grouped_df['도착 시군구 코드'].isin(top_regions)]
    # grouped_df_2 = grouped_df_2[grouped_df_2['도착 시군구 코드'].isin(top_regions)]
    grouped_df_3 = grouped_df_3[grouped_df_3['도착 시군구 코드'].isin(top_regions)]

    for region in top_regions:
        df_region = grouped_df[grouped_df['도착 시군구 코드'] == region]
        df_region_2 = grouped_df_2[grouped_df_2['도착 시군구 코드'] == region]
        df_region_3 = grouped_df_3[grouped_df_3['도착 시군구 코드'] == region]
        
        # 요일순서에 따라 데이터 정렬
        df_region_3['요일'] = pd.Categorical(df_region_3['요일'], categories=요일순서, ordered=True)
        df_region_3.sort_values(by='요일', inplace=True)
        
        # 서브 플롯 설정 (1x3)
        plt.figure(figsize=(18,6))

        # 원형그래프
        plt.subplot(1, 3, 1)
        plt.pie(df_region['이동인구(합)'], labels=df_region['성별'], autopct='%1.1f%%')
        plt.title(f'{region} - 성별 분포 ')
        
        # # 꺽은선그래프
        # plt.subplot(1, 3, 2)
        # df_region_2.set_index('나이')['이동인구(합)'].plot(kind='line', marker='o')
        # plt.title(f'{region} - 나이 ')
        
        # 막대그래프
        plt.subplot(1 , 3 , 2)
        df_region_3.set_index('요일')['이동인구(합)'].plot(kind='bar', color='skyblue')
        plt.title(f'{region} - 요일 ')

        # 꺽은선 그래프 & 막대그래프로 나이-성별재분류 추가 
        plt.subplot(1, 3, 2)
        df_region_age_gender = grouped_df_age_gender[grouped_df_age_gender['도착 시군구 코드'] == region].set_index('나이')
        df_region_age_gender.plot(kind='bar', stacked=True, figsize=(6, 4))  # 사이즈 조정
        df_region_age_gender.plot(figsize=(6, 4))   # 사이즈 조정
        plt.title(f'Age and Gender Distribution in {region}')
        plt.xlabel('Age')
        plt.ylabel('Count')
        plt.grid()
        
        # 그래프 출력
        plt.show()

# 시군구 기준 야간 시간  
## 1) 이동인구가 가장 많은 상위 10개  
## 2) 성별 분포
## 3) 요일 분포
## 4) 나이 기준으로 성별 분포  
### 야간 기준 ( 19 ~ 24시 사이)

In [23]:
    filtered_df_FE = df_2022_Q1[(df_2022_Q1['이동시간'] == '야간')]

    filtered_df_FE = filtered_df_FE.compute()

    sorted_df = filtered_df_FE.groupby('도착 시군구 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    plt.figure(figsize=(10,6))
    sorted_df.plot(kind='bar', color='skyblue')
    plt.title('야간 19 ~ 24시 전 도착 기준 시군구 코드별 이동인구 합 상위 10개')
    plt.xlabel('도착 시군구 코드')
    plt.ylabel('이동인구 합계')
    plt.show()

    요일순서 = ['월', '화', '수', '목', '금', '토', '일']

    # 도착 시군구 코드와 성별 나이 요일을 그룹화하고, 각 그룹의 이동인구(합)을 합산
    grouped_df = filtered_df_FE.groupby(['도착 시군구 코드', '성별'])['이동인구(합)'].sum().reset_index()
    grouped_df_2 = filtered_df_FE.groupby(['도착 시군구 코드', '나이'])['이동인구(합)'].sum().reset_index()
    grouped_df_3 = filtered_df_FE.groupby(['도착 시군구 코드', '요일'])['이동인구(합)'].sum().reset_index()
    grouped_df_age_gender = filtered_df_FE.groupby(['도착 시군구 코드', '나이', '성별'])['이동인구(합)'].sum().unstack().reset_index()


    # 상위 10개 도착 시군 구 코드만 선택
    top_regions = sorted_df.index.tolist()
    # grouped_df = grouped_df[grouped_df['도착 시군구 코드'].isin(top_regions)]
    # grouped_df_2 = grouped_df_2[grouped_df_2['도착 시군구 코드'].isin(top_regions)]
    grouped_df_3 = grouped_df_3[grouped_df_3['도착 시군구 코드'].isin(top_regions)]

    for region in top_regions:
        df_region = grouped_df[grouped_df['도착 시군구 코드'] == region]
        df_region_2 = grouped_df_2[grouped_df_2['도착 시군구 코드'] == region]
        df_region_3 = grouped_df_3[grouped_df_3['도착 시군구 코드'] == region]
        
        # 요일순서에 따라 데이터 정렬
        df_region_3['요일'] = pd.Categorical(df_region_3['요일'], categories=요일순서, ordered=True)
        df_region_3.sort_values(by='요일', inplace=True)
        
        # 서브 플롯 설정 (1x3)
        plt.figure(figsize=(18,6))

        # 원형그래프
        plt.subplot(1, 3, 1)
        plt.pie(df_region['이동인구(합)'], labels=df_region['성별'], autopct='%1.1f%%')
        plt.title(f'{region} - 성별 분포 ')
        
        # # 꺽은선그래프
        # plt.subplot(1, 3, 2)
        # df_region_2.set_index('나이')['이동인구(합)'].plot(kind='line', marker='o')
        # plt.title(f'{region} - 나이 ')
        
        # 막대그래프
        plt.subplot(1 , 3 , 2)
        df_region_3.set_index('요일')['이동인구(합)'].plot(kind='bar', color='skyblue')
        plt.title(f'{region} - 요일 ')

        # 꺽은선 그래프 & 막대그래프로 나이-성별재분류 추가 
        plt.subplot(1, 3, 2)
        df_region_age_gender = grouped_df_age_gender[grouped_df_age_gender['도착 시군구 코드'] == region].set_index('나이')
        df_region_age_gender.plot(kind='bar', stacked=True, figsize=(6, 4))  # 사이즈 조정
        df_region_age_gender.plot(figsize=(6, 4))   # 사이즈 조정
        plt.title(f'Age and Gender Distribution in {region}')
        plt.xlabel('Age')
        plt.ylabel('Count')
        plt.grid()
        
        # 그래프 출력
        plt.show()

# 기타 이동 기준 : 특정 요일에 가장 많이 방문하는 행정동 상위 10개 지역

In [24]:
# 요일 리스트 생성
days = ['월', '화', '수', '목', '금', '토', '일']

# 요일별로 상위 10개 도착지와 이동인구 합계 계산
top_dong_by_day = df_2022_Q1.groupby(['요일', '도착 행정동 코드'])['이동인구(합)'].sum().compute().groupby('요일').nlargest(10)

for day in days:
    # 특정 요일에 대한 데이터 추출
    df_day = top_dong_by_day[day].reset_index(level=0, drop=True)
    
    # 막대 그래프
    plt.figure(figsize=(15, 10))
    plt.subplot(2,2,1)
    sns.barplot(x=df_day.index, y=df_day.values)
    plt.title(f"{day}요일에 가장 많이 방문하는 행정동 상위 10곳")
    
    # 원형 그래프
    plt.subplot(2,2,2)
    plt.pie(df_day.values,
            labels=df_day.index,
            autopct='%1.1f%%')
    
   # 꺽은선 그래프 (선그래프는 순서가 중요하기 때문에 정렬된 값으로 생성)
    df_sorted = df_day.sort_index()
    plt.subplot(2,2,3)
    sns.lineplot(x=df_sorted.index,y=df_sorted.values)

   # 레이아웃 조절 및 차트 출력
    plt.tight_layout()
    plt.show()

# 행정동 기준 심야 시간  
## 1) 이동인구가 가장 많은 상위 10개  
## 2) 성별 분포
## 3) 요일 분포
## 4) 나이 기준으로 성별 분포
### 심야 기준 ( 0 ~ 6시 사이)

In [25]:
    filtered_df_AE = df_2022_Q1[(df_2022_Q1['이동시간'] == '심야')]

    filtered_df_AE = filtered_df_AE.compute()

    sorted_df = filtered_df_AE.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    plt.figure(figsize=(10,6))
    sorted_df.plot(kind='bar', color='skyblue')
    plt.title('심야 0 ~ 6시 전 도착 기준 행정동 코드별 이동인구 합 상위 10개')
    plt.xlabel('도착 행정동 코드')
    plt.ylabel('이동인구 합계')
    plt.show()

    요일순서 = ['월', '화', '수', '목', '금', '토', '일']

    # 도착 행정동 코드와 성별 나이 요일을 그룹화하고, 각 그룹의 이동인구(합)을 합산
    grouped_df = filtered_df_AE.groupby(['도착 행정동 코드', '성별'])['이동인구(합)'].sum().reset_index()
    grouped_df_2 = filtered_df_AE.groupby(['도착 행정동 코드', '나이'])['이동인구(합)'].sum().reset_index()
    grouped_df_3 = filtered_df_AE.groupby(['도착 행정동 코드', '요일'])['이동인구(합)'].sum().reset_index()
    grouped_df_age_gender = filtered_df_AE.groupby(['도착 행정동 코드', '나이', '성별'])['이동인구(합)'].sum().unstack().reset_index()


    # 상위 10개 도착 시군 구 코드만 선택
    top_regions = sorted_df.index.tolist()
    # grouped_df = grouped_df[grouped_df['도착 행정동 코드'].isin(top_regions)]
    # grouped_df_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'].isin(top_regions)]
    grouped_df_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'].isin(top_regions)]

    for region in top_regions:
        df_region = grouped_df[grouped_df['도착 행정동 코드'] == region]
        df_region_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'] == region]
        df_region_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'] == region]
        
        # 요일순서에 따라 데이터 정렬
        df_region_3['요일'] = pd.Categorical(df_region_3['요일'], categories=요일순서, ordered=True)
        df_region_3.sort_values(by='요일', inplace=True)
        
        # 서브 플롯 설정 (1x3)
        plt.figure(figsize=(18,6))

        # 원형그래프
        plt.subplot(1, 3, 1)
        plt.pie(df_region['이동인구(합)'], labels=df_region['성별'], autopct='%1.1f%%')
        plt.title(f'{region} - 성별 분포 ')
        
        # # 꺽은선그래프
        # plt.subplot(1, 3, 2)
        # df_region_2.set_index('나이')['이동인구(합)'].plot(kind='line', marker='o')
        # plt.title(f'{region} - 나이 ')
        
        # 막대그래프
        plt.subplot(1 , 3 , 2)
        df_region_3.set_index('요일')['이동인구(합)'].plot(kind='bar', color='skyblue')
        plt.title(f'{region} - 요일 ')

        # 꺽은선 그래프 & 막대그래프로 나이-성별재분류 추가 
        plt.subplot(1, 3, 2)
        df_region_age_gender = grouped_df_age_gender[grouped_df_age_gender['도착 행정동 코드'] == region].set_index('나이')
        df_region_age_gender.plot(kind='bar', stacked=True, figsize=(6, 4))  # 사이즈 조정
        df_region_age_gender.plot(figsize=(6, 4))   # 사이즈 조정
        plt.title(f'Age and Gender Distribution in {region}')
        plt.xlabel('Age')
        plt.ylabel('Count')
        plt.grid()
        
        # 그래프 출력
        plt.show()

# 행정동 기준 오전 시간  
## 1) 이동인구가 가장 많은 상위 10개  
## 2) 성별 분포
## 3) 요일 분포
## 4) 나이 기준으로 성별 분포 
### 오전 기준( 6 ~ 11시 사이)

In [26]:
    filtered_df_BE = df_2022_Q1[(df_2022_Q1['이동시간'] == '오전')]

    filtered_df_BE = filtered_df_BE.compute()

    sorted_df = filtered_df_BE.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    plt.figure(figsize=(10,6))
    sorted_df.plot(kind='bar', color='skyblue')
    plt.title('오전 6 ~ 11시 전 도착 기준 행정동 코드별 이동인구 합 상위 10개')
    plt.xlabel('도착 행정동 코드')
    plt.ylabel('이동인구 합계')
    plt.show()

    요일순서 = ['월', '화', '수', '목', '금', '토', '일']

    # 도착 행정동 코드와 성별 나이 요일을 그룹화하고, 각 그룹의 이동인구(합)을 합산
    grouped_df = filtered_df_BE.groupby(['도착 행정동 코드', '성별'])['이동인구(합)'].sum().reset_index()
    grouped_df_2 = filtered_df_BE.groupby(['도착 행정동 코드', '나이'])['이동인구(합)'].sum().reset_index()
    grouped_df_3 = filtered_df_BE.groupby(['도착 행정동 코드', '요일'])['이동인구(합)'].sum().reset_index()
    grouped_df_age_gender = filtered_df_BE.groupby(['도착 행정동 코드', '나이', '성별'])['이동인구(합)'].sum().unstack().reset_index()


    # 상위 10개 도착 시군 구 코드만 선택
    top_regions = sorted_df.index.tolist()
    # grouped_df = grouped_df[grouped_df['도착 행정동 코드'].isin(top_regions)]
    # grouped_df_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'].isin(top_regions)]
    grouped_df_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'].isin(top_regions)]

    for region in top_regions:
        df_region = grouped_df[grouped_df['도착 행정동 코드'] == region]
        df_region_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'] == region]
        df_region_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'] == region]
        
        # 요일순서에 따라 데이터 정렬
        df_region_3['요일'] = pd.Categorical(df_region_3['요일'], categories=요일순서, ordered=True)
        df_region_3.sort_values(by='요일', inplace=True)
        
        # 서브 플롯 설정 (1x3)
        plt.figure(figsize=(18,6))

        # 원형그래프
        plt.subplot(1, 3, 1)
        plt.pie(df_region['이동인구(합)'], labels=df_region['성별'], autopct='%1.1f%%')
        plt.title(f'{region} - 성별 분포 ')
        
        # # 꺽은선그래프
        # plt.subplot(1, 3, 2)
        # df_region_2.set_index('나이')['이동인구(합)'].plot(kind='line', marker='o')
        # plt.title(f'{region} - 나이 ')
        
        # 막대그래프
        plt.subplot(1 , 3 , 2)
        df_region_3.set_index('요일')['이동인구(합)'].plot(kind='bar', color='skyblue')
        plt.title(f'{region} - 요일 ')

        # 꺽은선 그래프 & 막대그래프로 나이-성별재분류 추가 
        plt.subplot(1, 3, 2)
        df_region_age_gender = grouped_df_age_gender[grouped_df_age_gender['도착 행정동 코드'] == region].set_index('나이')
        df_region_age_gender.plot(kind='bar', stacked=True, figsize=(6, 4))  # 사이즈 조정
        df_region_age_gender.plot(figsize=(6, 4))   # 사이즈 조정
        plt.title(f'Age and Gender Distribution in {region}')
        plt.xlabel('Age')
        plt.ylabel('Count')
        plt.grid()
        
        # 그래프 출력
        plt.show()

# 행정동 기준 점심 시간  
## 1) 이동인구가 가장 많은 상위 10개  
## 2) 성별 분포
## 3) 요일 분포
## 4) 나이 기준으로 성별 분포
### 점심 기준 ( 11 ~ 13시 사이)

In [27]:
    filtered_df_CE = df_2022_Q1[(df_2022_Q1['이동시간'] == '점심')]

    filtered_df_CE = filtered_df_CE.compute()

    sorted_df = filtered_df_CE.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    plt.figure(figsize=(10,6))
    sorted_df.plot(kind='bar', color='skyblue')
    plt.title('점심 11 ~ 13시 전 도착 기준 행정동 코드별 이동인구 합 상위 10개')
    plt.xlabel('도착 행정동 코드')
    plt.ylabel('이동인구 합계')
    plt.show()

    요일순서 = ['월', '화', '수', '목', '금', '토', '일']

    # 도착 행정동 코드와 성별 나이 요일을 그룹화하고, 각 그룹의 이동인구(합)을 합산
    grouped_df = filtered_df_CE.groupby(['도착 행정동 코드', '성별'])['이동인구(합)'].sum().reset_index()
    grouped_df_2 = filtered_df_CE.groupby(['도착 행정동 코드', '나이'])['이동인구(합)'].sum().reset_index()
    grouped_df_3 = filtered_df_CE.groupby(['도착 행정동 코드', '요일'])['이동인구(합)'].sum().reset_index()
    grouped_df_age_gender = filtered_df_CE.groupby(['도착 행정동 코드', '나이', '성별'])['이동인구(합)'].sum().unstack().reset_index()


    # 상위 10개 도착 시군 구 코드만 선택
    top_regions = sorted_df.index.tolist()
    # grouped_df = grouped_df[grouped_df['도착 행정동 코드'].isin(top_regions)]
    # grouped_df_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'].isin(top_regions)]
    grouped_df_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'].isin(top_regions)]

    for region in top_regions:
        df_region = grouped_df[grouped_df['도착 행정동 코드'] == region]
        df_region_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'] == region]
        df_region_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'] == region]
        
        # 요일순서에 따라 데이터 정렬
        df_region_3['요일'] = pd.Categorical(df_region_3['요일'], categories=요일순서, ordered=True)
        df_region_3.sort_values(by='요일', inplace=True)
        
        # 서브 플롯 설정 (1x3)
        plt.figure(figsize=(18,6))

        # 원형그래프
        plt.subplot(1, 3, 1)
        plt.pie(df_region['이동인구(합)'], labels=df_region['성별'], autopct='%1.1f%%')
        plt.title(f'{region} - 성별 분포 ')
        
        # # 꺽은선그래프
        # plt.subplot(1, 3, 2)
        # df_region_2.set_index('나이')['이동인구(합)'].plot(kind='line', marker='o')
        # plt.title(f'{region} - 나이 ')
        
        # 막대그래프
        plt.subplot(1 , 3 , 2)
        df_region_3.set_index('요일')['이동인구(합)'].plot(kind='bar', color='skyblue')
        plt.title(f'{region} - 요일 ')

        # 꺽은선 그래프 & 막대그래프로 나이-성별재분류 추가 
        plt.subplot(1, 3, 2)
        df_region_age_gender = grouped_df_age_gender[grouped_df_age_gender['도착 행정동 코드'] == region].set_index('나이')
        df_region_age_gender.plot(kind='bar', stacked=True, figsize=(6, 4))  # 사이즈 조정
        df_region_age_gender.plot(figsize=(6, 4))   # 사이즈 조정
        plt.title(f'Age and Gender Distribution in {region}')
        plt.xlabel('Age')
        plt.ylabel('Count')
        plt.grid()
        
        # 그래프 출력
        plt.show()

# 행정동 기준 오후 시간  
## 1) 이동인구가 가장 많은 상위 10개  
## 2) 성별 분포
## 3) 요일 분포
## 4) 나이 기준으로 성별 분포 
### 오후 기준 ( 13 ~ 17시 사이)

In [28]:
    filtered_df_DE = df_2022_Q1[(df_2022_Q1['이동시간'] == '오후')]

    filtered_df_DE = filtered_df_DE.compute()

    sorted_df = filtered_df_DE.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    plt.figure(figsize=(10,6))
    sorted_df.plot(kind='bar', color='skyblue')
    plt.title('오후 13 ~ 17시 전 도착 기준 행정동 코드별 이동인구 합 상위 10개')
    plt.xlabel('도착 행정동 코드')
    plt.ylabel('이동인구 합계')
    plt.show()

    요일순서 = ['월', '화', '수', '목', '금', '토', '일']

    # 도착 행정동 코드와 성별 나이 요일을 그룹화하고, 각 그룹의 이동인구(합)을 합산
    grouped_df = filtered_df_DE.groupby(['도착 행정동 코드', '성별'])['이동인구(합)'].sum().reset_index()
    grouped_df_2 = filtered_df_DE.groupby(['도착 행정동 코드', '나이'])['이동인구(합)'].sum().reset_index()
    grouped_df_3 = filtered_df_DE.groupby(['도착 행정동 코드', '요일'])['이동인구(합)'].sum().reset_index()
    grouped_df_age_gender = filtered_df_DE.groupby(['도착 행정동 코드', '나이', '성별'])['이동인구(합)'].sum().unstack().reset_index()


    # 상위 10개 도착 시군 구 코드만 선택
    top_regions = sorted_df.index.tolist()
    # grouped_df = grouped_df[grouped_df['도착 행정동 코드'].isin(top_regions)]
    # grouped_df_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'].isin(top_regions)]
    grouped_df_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'].isin(top_regions)]

    for region in top_regions:
        df_region = grouped_df[grouped_df['도착 행정동 코드'] == region]
        df_region_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'] == region]
        df_region_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'] == region]
        
        # 요일순서에 따라 데이터 정렬
        df_region_3['요일'] = pd.Categorical(df_region_3['요일'], categories=요일순서, ordered=True)
        df_region_3.sort_values(by='요일', inplace=True)
        
        # 서브 플롯 설정 (1x3)
        plt.figure(figsize=(18,6))

        # 원형그래프
        plt.subplot(1, 3, 1)
        plt.pie(df_region['이동인구(합)'], labels=df_region['성별'], autopct='%1.1f%%')
        plt.title(f'{region} - 성별 분포 ')
        
        # # 꺽은선그래프
        # plt.subplot(1, 3, 2)
        # df_region_2.set_index('나이')['이동인구(합)'].plot(kind='line', marker='o')
        # plt.title(f'{region} - 나이 ')
        
        # 막대그래프
        plt.subplot(1 , 3 , 2)
        df_region_3.set_index('요일')['이동인구(합)'].plot(kind='bar', color='skyblue')
        plt.title(f'{region} - 요일 ')

        # 꺽은선 그래프 & 막대그래프로 나이-성별재분류 추가 
        plt.subplot(1, 3, 2)
        df_region_age_gender = grouped_df_age_gender[grouped_df_age_gender['도착 행정동 코드'] == region].set_index('나이')
        df_region_age_gender.plot(kind='bar', stacked=True, figsize=(6, 4))  # 사이즈 조정
        df_region_age_gender.plot(figsize=(6, 4))   # 사이즈 조정
        plt.title(f'Age and Gender Distribution in {region}')
        plt.xlabel('Age')
        plt.ylabel('Count')
        plt.grid()
        
        # 그래프 출력
        plt.show()

# 행정동 기준 저녁 시간  
## 1) 이동인구가 가장 많은 상위 10개  
## 2) 성별 분포
## 3) 요일 분포
## 4) 나이 기준으로 성별 분포 
### 저녁 기준 ( 17 ~ 19시 사이)

In [29]:
    filtered_df_EE = df_2022_Q1[(df_2022_Q1['이동시간'] == '저녁')]

    filtered_df_EE = filtered_df_EE.compute()

    sorted_df = filtered_df_EE.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    plt.figure(figsize=(10,6))
    sorted_df.plot(kind='bar', color='skyblue')
    plt.title('저녁 17 ~ 19시 전 도착 기준 행정동 코드별 이동인구 합 상위 10개')
    plt.xlabel('도착 행정동 코드')
    plt.ylabel('이동인구 합계')
    plt.show()

    요일순서 = ['월', '화', '수', '목', '금', '토', '일']

    # 도착 행정동 코드와 성별 나이 요일을 그룹화하고, 각 그룹의 이동인구(합)을 합산
    grouped_df = filtered_df_EE.groupby(['도착 행정동 코드', '성별'])['이동인구(합)'].sum().reset_index()
    grouped_df_2 = filtered_df_EE.groupby(['도착 행정동 코드', '나이'])['이동인구(합)'].sum().reset_index()
    grouped_df_3 = filtered_df_EE.groupby(['도착 행정동 코드', '요일'])['이동인구(합)'].sum().reset_index()
    grouped_df_age_gender = filtered_df_EE.groupby(['도착 행정동 코드', '나이', '성별'])['이동인구(합)'].sum().unstack().reset_index()


    # 상위 10개 도착 시군 구 코드만 선택
    top_regions = sorted_df.index.tolist()
    # grouped_df = grouped_df[grouped_df['도착 행정동 코드'].isin(top_regions)]
    # grouped_df_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'].isin(top_regions)]
    grouped_df_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'].isin(top_regions)]

    for region in top_regions:
        df_region = grouped_df[grouped_df['도착 행정동 코드'] == region]
        df_region_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'] == region]
        df_region_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'] == region]
        
        # 요일순서에 따라 데이터 정렬
        df_region_3['요일'] = pd.Categorical(df_region_3['요일'], categories=요일순서, ordered=True)
        df_region_3.sort_values(by='요일', inplace=True)
        
        # 서브 플롯 설정 (1x3)
        plt.figure(figsize=(18,6))

        # 원형그래프
        plt.subplot(1, 3, 1)
        plt.pie(df_region['이동인구(합)'], labels=df_region['성별'], autopct='%1.1f%%')
        plt.title(f'{region} - 성별 분포 ')
        
        # # 꺽은선그래프
        # plt.subplot(1, 3, 2)
        # df_region_2.set_index('나이')['이동인구(합)'].plot(kind='line', marker='o')
        # plt.title(f'{region} - 나이 ')
        
        # 막대그래프
        plt.subplot(1 , 3 , 2)
        df_region_3.set_index('요일')['이동인구(합)'].plot(kind='bar', color='skyblue')
        plt.title(f'{region} - 요일 ')

        # 꺽은선 그래프 & 막대그래프로 나이-성별재분류 추가 
        plt.subplot(1, 3, 2)
        df_region_age_gender = grouped_df_age_gender[grouped_df_age_gender['도착 행정동 코드'] == region].set_index('나이')
        df_region_age_gender.plot(kind='bar', stacked=True, figsize=(6, 4))  # 사이즈 조정
        df_region_age_gender.plot(figsize=(6, 4))   # 사이즈 조정
        plt.title(f'Age and Gender Distribution in {region}')
        plt.xlabel('Age')
        plt.ylabel('Count')
        plt.grid()
        
        # 그래프 출력
        plt.show()

# 행정구 기준 야간 시간  
## 1) 이동인구가 가장 많은 상위 10개  
## 2) 성별 분포
## 3) 요일 분포
## 4) 나이 기준으로 성별 분포  
### 야간 기준 ( 19 ~ 24시 사이)

In [30]:
    filtered_df_FE = df_2022_Q1[(df_2022_Q1['이동시간'] == '야간')]

    filtered_df_FE = filtered_df_FE.compute()

    sorted_df = filtered_df_FE.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    plt.figure(figsize=(10,6))
    sorted_df.plot(kind='bar', color='skyblue')
    plt.title('야간 19 ~ 24시 전 도착 기준 행정동 코드별 이동인구 합 상위 10개')
    plt.xlabel('도착 행정동 코드')
    plt.ylabel('이동인구 합계')
    plt.show()

    요일순서 = ['월', '화', '수', '목', '금', '토', '일']

    # 도착 행정동 코드와 성별 나이 요일을 그룹화하고, 각 그룹의 이동인구(합)을 합산
    grouped_df = filtered_df_FE.groupby(['도착 행정동 코드', '성별'])['이동인구(합)'].sum().reset_index()
    grouped_df_2 = filtered_df_FE.groupby(['도착 행정동 코드', '나이'])['이동인구(합)'].sum().reset_index()
    grouped_df_3 = filtered_df_FE.groupby(['도착 행정동 코드', '요일'])['이동인구(합)'].sum().reset_index()
    grouped_df_age_gender = filtered_df_FE.groupby(['도착 행정동 코드', '나이', '성별'])['이동인구(합)'].sum().unstack().reset_index()


    # 상위 10개 도착 시군 구 코드만 선택
    top_regions = sorted_df.index.tolist()
    # grouped_df = grouped_df[grouped_df['도착 행정동 코드'].isin(top_regions)]
    # grouped_df_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'].isin(top_regions)]
    grouped_df_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'].isin(top_regions)]

    for region in top_regions:
        df_region = grouped_df[grouped_df['도착 행정동 코드'] == region]
        df_region_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'] == region]
        df_region_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'] == region]
        
        # 요일순서에 따라 데이터 정렬
        df_region_3['요일'] = pd.Categorical(df_region_3['요일'], categories=요일순서, ordered=True)
        df_region_3.sort_values(by='요일', inplace=True)
        
        # 서브 플롯 설정 (1x3)
        plt.figure(figsize=(18,6))

        # 원형그래프
        plt.subplot(1, 3, 1)
        plt.pie(df_region['이동인구(합)'], labels=df_region['성별'], autopct='%1.1f%%')
        plt.title(f'{region} - 성별 분포 ')
        
        # # 꺽은선그래프
        # plt.subplot(1, 3, 2)
        # df_region_2.set_index('나이')['이동인구(합)'].plot(kind='line', marker='o')
        # plt.title(f'{region} - 나이 ')
        
        # 막대그래프
        plt.subplot(1 , 3 , 2)
        df_region_3.set_index('요일')['이동인구(합)'].plot(kind='bar', color='skyblue')
        plt.title(f'{region} - 요일 ')

        # 꺽은선 그래프 & 막대그래프로 나이-성별재분류 추가 
        plt.subplot(1, 3, 2)
        df_region_age_gender = grouped_df_age_gender[grouped_df_age_gender['도착 행정동 코드'] == region].set_index('나이')
        df_region_age_gender.plot(kind='bar', stacked=True, figsize=(6, 4))  # 사이즈 조정
        df_region_age_gender.plot(figsize=(6, 4))   # 사이즈 조정
        plt.title(f'Age and Gender Distribution in {region}')
        plt.xlabel('Age')
        plt.ylabel('Count')
        plt.grid()
        
        # 그래프 출력
        plt.show()

# 서울 시 기준 사람들이 많이 놀러 갈곳으로 예상되는  11곳 비교
## 강남구, 광진구, 마포구, 성동구, 송파구 서초구,  영등포구, 용산, 중구, 종로구, 성동구

## 강남구에 가장 많이 방문하는 행정동의 성별, 나이 및 세대 별 성별 비교

In [31]:
    # 서울 시 기준 사람들이 많이 놀러 갈곳으로 예상되는  10곳 비교
    ## 강남구, 광진구, 마포구, 성동구, 송파구 서초구,  영등포구, 용산, 중구, 종로구, 성동구


    # 강남구는 도착 시군구 코드가 '강남구'
    gangnam_code = "강남구"

    # 강남구 데이터 필터링
    df_gangnam_G = df_2022_Q1[df_2022_Q1['도착 시군구 코드'] == gangnam_code]

    # 도착 행정동 별로 그룹화하여 이동인구(합) 합계 계산
    df_gangnam_G_grouped = df_gangnam_G.groupby('도착 행정동 코드')['이동인구(합)'].sum().nlargest(10)
    #df_gangnam_G_grouped = df_gangnam_G.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    # pandas DataFrame으로 변환
    df_gangnam_G_grouped = df_gangnam_G_grouped.compute()

    # 원 그래프
    plt.subplot(1,3,2)
    plt.pie(df_gangnam_G_grouped.values,
            labels=df_gangnam_G_grouped.index,
            autopct='%1.1f%%')
    plt.title("강남구 기준 가장 많이 방문하는 행정동 상위 10개")

    # 레이아웃 조절 및 차트 출력
    plt.tight_layout()
    plt.show()


In [32]:
    filtered_df_D = df_2022_Q1[(df_2022_Q1['도착 시군구 코드'] == '강남구')]

    filtered_df_D = filtered_df_D.compute()

    sorted_df = filtered_df_D.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    plt.figure(figsize=(10,6))
    sorted_df.plot(kind='bar', color='skyblue')
    plt.title('강남구 내의 행정동 유동인구 상위 10개')
    plt.xlabel('도착 행정동 코드')
    plt.ylabel('이동인구 합계')
    plt.show()

    요일순서 = ['월', '화', '수', '목', '금', '토', '일']

    # 도착 행정동 코드와 성별 나이 요일을 그룹화하고, 각 그룹의 이동인구(합)을 합산
    grouped_df = filtered_df_D.groupby(['도착 행정동 코드', '성별'])['이동인구(합)'].sum().reset_index()
    grouped_df_2 = filtered_df_D.groupby(['도착 행정동 코드', '나이'])['이동인구(합)'].sum().reset_index()
    grouped_df_3 = filtered_df_D.groupby(['도착 행정동 코드', '요일'])['이동인구(합)'].sum().reset_index()
    grouped_df_age_gender = filtered_df_D.groupby(['도착 행정동 코드', '나이', '성별'])['이동인구(합)'].sum().unstack().reset_index()


    # 상위 10개 도착 시군 구 코드만 선택
    top_regions = sorted_df.index.tolist()
    # grouped_df = grouped_df[grouped_df['도착 행정동 코드'].isin(top_regions)]
    # grouped_df_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'].isin(top_regions)]
    grouped_df_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'].isin(top_regions)]

    for region in top_regions:
        df_region = grouped_df[grouped_df['도착 행정동 코드'] == region]
        df_region_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'] == region]
        df_region_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'] == region]
        
        # 요일순서에 따라 데이터 정렬
        df_region_3['요일'] = pd.Categorical(df_region_3['요일'], categories=요일순서, ordered=True)
        df_region_3.sort_values(by='요일', inplace=True)
        
        # 서브 플롯 설정 (1x3)
        plt.figure(figsize=(18,6))

        # 원형그래프
        plt.subplot(1, 3, 1)
        plt.pie(df_region['이동인구(합)'], labels=df_region['성별'], autopct='%1.1f%%')
        plt.title(f'{region} - 성별 분포 ')
        
        # # 꺽은선그래프
        # plt.subplot(1, 3, 2)
        # df_region_2.set_index('나이')['이동인구(합)'].plot(kind='line', marker='o')
        # plt.title(f'{region} - 나이 ')
        
        # 막대그래프
        plt.subplot(1 , 3 , 2)
        df_region_3.set_index('요일')['이동인구(합)'].plot(kind='bar', color='skyblue')
        plt.title(f'{region} - 요일 ')

        # 꺽은선 그래프 & 막대그래프로 나이-성별재분류 추가 
        plt.subplot(1, 3, 2)
        df_region_age_gender = grouped_df_age_gender[grouped_df_age_gender['도착 행정동 코드'] == region].set_index('나이')
        # df_region_age_gender.plot(kind='bar', stacked=True, figsize=(6, 4))  # 사이즈 조정
        df_region_age_gender.plot(figsize=(6, 4))   # 사이즈 조정
        plt.title(f'Age and Gender Distribution in {region}')
        plt.xlabel('Age')
        plt.ylabel('Count')
        plt.grid()
        
        # 그래프 출력
        plt.show()

## 광진구에 가장 많이 방문하는 행정동의 성별, 나이 및 세대 별 성별 비교

In [33]:
    # 강남구는 도착 시군구 코드가 '강남구'
    gangnam_code = "광진구"

    # 강남구 데이터 필터링
    df_gangnam_G = df_2022_Q1[df_2022_Q1['도착 시군구 코드'] == gangnam_code]

    # 도착 행정동 별로 그룹화하여 이동인구(합) 합계 계산
    df_gangnam_G_grouped = df_gangnam_G.groupby('도착 행정동 코드')['이동인구(합)'].sum().nlargest(10)
    #df_gangnam_G_grouped = df_gangnam_G.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    # pandas DataFrame으로 변환
    df_gangnam_G_grouped = df_gangnam_G_grouped.compute()

    # 원 그래프
    plt.subplot(1,3,2)
    plt.pie(df_gangnam_G_grouped.values,
            labels=df_gangnam_G_grouped.index,
            autopct='%1.1f%%')
    plt.title("광진구 기준 가장 많이 방문하는 행정동 상위 10개")

    # 레이아웃 조절 및 차트 출력
    plt.tight_layout()
    plt.show()

    filtered_df_D = df_2022_Q1[(df_2022_Q1['도착 시군구 코드'] == '광진구')]

    filtered_df_D = filtered_df_D.compute()

    sorted_df = filtered_df_D.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    plt.figure(figsize=(10,6))
    sorted_df.plot(kind='bar', color='skyblue')
    plt.title('광진구 내의 행정동 유동인구 상위 10개')
    plt.xlabel('도착 행정동 코드')
    plt.ylabel('이동인구 합계')
    plt.show()

    요일순서 = ['월', '화', '수', '목', '금', '토', '일']

    # 도착 행정동 코드와 성별 나이 요일을 그룹화하고, 각 그룹의 이동인구(합)을 합산
    grouped_df = filtered_df_D.groupby(['도착 행정동 코드', '성별'])['이동인구(합)'].sum().reset_index()
    grouped_df_2 = filtered_df_D.groupby(['도착 행정동 코드', '나이'])['이동인구(합)'].sum().reset_index()
    grouped_df_3 = filtered_df_D.groupby(['도착 행정동 코드', '요일'])['이동인구(합)'].sum().reset_index()
    grouped_df_age_gender = filtered_df_D.groupby(['도착 행정동 코드', '나이', '성별'])['이동인구(합)'].sum().unstack().reset_index()


    # 상위 10개 도착 시군 구 코드만 선택
    top_regions = sorted_df.index.tolist()
    # grouped_df = grouped_df[grouped_df['도착 행정동 코드'].isin(top_regions)]
    # grouped_df_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'].isin(top_regions)]
    grouped_df_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'].isin(top_regions)]

    for region in top_regions:
        df_region = grouped_df[grouped_df['도착 행정동 코드'] == region]
        df_region_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'] == region]
        df_region_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'] == region]
        
        # 요일순서에 따라 데이터 정렬
        df_region_3['요일'] = pd.Categorical(df_region_3['요일'], categories=요일순서, ordered=True)
        df_region_3.sort_values(by='요일', inplace=True)
        
        # 서브 플롯 설정 (1x3)
        plt.figure(figsize=(18,6))

        # 원형그래프
        plt.subplot(1, 3, 1)
        plt.pie(df_region['이동인구(합)'], labels=df_region['성별'], autopct='%1.1f%%')
        plt.title(f'{region} - 성별 분포 ')
        
        # # 꺽은선그래프
        # plt.subplot(1, 3, 2)
        # df_region_2.set_index('나이')['이동인구(합)'].plot(kind='line', marker='o')
        # plt.title(f'{region} - 나이 ')
        
        # 막대그래프
        plt.subplot(1 , 3 , 2)
        df_region_3.set_index('요일')['이동인구(합)'].plot(kind='bar', color='skyblue')
        plt.title(f'{region} - 요일 ')

        # 꺽은선 그래프 & 막대그래프로 나이-성별재분류 추가 
        plt.subplot(1, 3, 2)
        df_region_age_gender = grouped_df_age_gender[grouped_df_age_gender['도착 행정동 코드'] == region].set_index('나이')
        # df_region_age_gender.plot(kind='bar', stacked=True, figsize=(6, 4))  # 사이즈 조정
        df_region_age_gender.plot(figsize=(6, 4))   # 사이즈 조정
        plt.title(f'Age and Gender Distribution in {region}')
        plt.xlabel('Age')
        plt.ylabel('Count')
        plt.grid()
        
        # 그래프 출력
        plt.show()


## 마포구에 가장 많이 방문하는 행정동의 성별, 나이 및 세대 별 성별 비교

In [34]:
    # 강남구는 도착 시군구 코드가 '강남구'
    gangnam_code = "마포구"

    # 강남구 데이터 필터링
    df_gangnam_G = df_2022_Q1[df_2022_Q1['도착 시군구 코드'] == gangnam_code]

    # 도착 행정동 별로 그룹화하여 이동인구(합) 합계 계산
    df_gangnam_G_grouped = df_gangnam_G.groupby('도착 행정동 코드')['이동인구(합)'].sum().nlargest(10)
    #df_gangnam_G_grouped = df_gangnam_G.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    # pandas DataFrame으로 변환
    df_gangnam_G_grouped = df_gangnam_G_grouped.compute()

    # 원 그래프
    plt.subplot(1,3,2)
    plt.pie(df_gangnam_G_grouped.values,
            labels=df_gangnam_G_grouped.index,
            autopct='%1.1f%%')
    plt.title("마포구 기준 가장 많이 방문하는 행정동 상위 10개")

    # 레이아웃 조절 및 차트 출력
    plt.tight_layout()
    plt.show()

    filtered_df_D = df_2022_Q1[(df_2022_Q1['도착 시군구 코드'] == '마포구')]

    filtered_df_D = filtered_df_D.compute()

    sorted_df = filtered_df_D.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    plt.figure(figsize=(10,6))
    sorted_df.plot(kind='bar', color='skyblue')
    plt.title('마포구 내의 행정동 유동인구 상위 10개')
    plt.xlabel('도착 행정동 코드')
    plt.ylabel('이동인구 합계')
    plt.show()

    요일순서 = ['월', '화', '수', '목', '금', '토', '일']

    # 도착 행정동 코드와 성별 나이 요일을 그룹화하고, 각 그룹의 이동인구(합)을 합산
    grouped_df = filtered_df_D.groupby(['도착 행정동 코드', '성별'])['이동인구(합)'].sum().reset_index()
    grouped_df_2 = filtered_df_D.groupby(['도착 행정동 코드', '나이'])['이동인구(합)'].sum().reset_index()
    grouped_df_3 = filtered_df_D.groupby(['도착 행정동 코드', '요일'])['이동인구(합)'].sum().reset_index()
    grouped_df_age_gender = filtered_df_D.groupby(['도착 행정동 코드', '나이', '성별'])['이동인구(합)'].sum().unstack().reset_index()


    # 상위 10개 도착 시군 구 코드만 선택
    top_regions = sorted_df.index.tolist()
    # grouped_df = grouped_df[grouped_df['도착 행정동 코드'].isin(top_regions)]
    # grouped_df_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'].isin(top_regions)]
    grouped_df_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'].isin(top_regions)]

    for region in top_regions:
        df_region = grouped_df[grouped_df['도착 행정동 코드'] == region]
        df_region_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'] == region]
        df_region_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'] == region]
        
        # 요일순서에 따라 데이터 정렬
        df_region_3['요일'] = pd.Categorical(df_region_3['요일'], categories=요일순서, ordered=True)
        df_region_3.sort_values(by='요일', inplace=True)
        
        # 서브 플롯 설정 (1x3)
        plt.figure(figsize=(18,6))

        # 원형그래프
        plt.subplot(1, 3, 1)
        plt.pie(df_region['이동인구(합)'], labels=df_region['성별'], autopct='%1.1f%%')
        plt.title(f'{region} - 성별 분포 ')
        
        # # 꺽은선그래프
        # plt.subplot(1, 3, 2)
        # df_region_2.set_index('나이')['이동인구(합)'].plot(kind='line', marker='o')
        # plt.title(f'{region} - 나이 ')
        
        # 막대그래프
        plt.subplot(1 , 3 , 2)
        df_region_3.set_index('요일')['이동인구(합)'].plot(kind='bar', color='skyblue')
        plt.title(f'{region} - 요일 ')

        # 꺽은선 그래프 & 막대그래프로 나이-성별재분류 추가 
        plt.subplot(1, 3, 2)
        df_region_age_gender = grouped_df_age_gender[grouped_df_age_gender['도착 행정동 코드'] == region].set_index('나이')
        # df_region_age_gender.plot(kind='bar', stacked=True, figsize=(6, 4))  # 사이즈 조정
        df_region_age_gender.plot(figsize=(6, 4))   # 사이즈 조정
        plt.title(f'Age and Gender Distribution in {region}')
        plt.xlabel('Age')
        plt.ylabel('Count')
        plt.grid()
        
        # 그래프 출력
        plt.show()


## 성동구에 가장 많이 방문하는 행정동의 성별, 나이 및 세대 별 성별 비교

In [35]:
    # 강남구는 도착 시군구 코드가 '강남구'
    gangnam_code = "성동구"

    # 강남구 데이터 필터링
    df_gangnam_G = df_2022_Q1[df_2022_Q1['도착 시군구 코드'] == gangnam_code]

    # 도착 행정동 별로 그룹화하여 이동인구(합) 합계 계산
    df_gangnam_G_grouped = df_gangnam_G.groupby('도착 행정동 코드')['이동인구(합)'].sum().nlargest(10)
    #df_gangnam_G_grouped = df_gangnam_G.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    # pandas DataFrame으로 변환
    df_gangnam_G_grouped = df_gangnam_G_grouped.compute()

    # 원 그래프
    plt.subplot(1,3,2)
    plt.pie(df_gangnam_G_grouped.values,
            labels=df_gangnam_G_grouped.index,
            autopct='%1.1f%%')
    plt.title("성동구 기준 가장 많이 방문하는 행정동 상위 10개")

    # 레이아웃 조절 및 차트 출력
    plt.tight_layout()
    plt.show()

    filtered_df_D = df_2022_Q1[(df_2022_Q1['도착 시군구 코드'] == '성동구')]

    filtered_df_D = filtered_df_D.compute()

    sorted_df = filtered_df_D.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    plt.figure(figsize=(10,6))
    sorted_df.plot(kind='bar', color='skyblue')
    plt.title('성동구 내의 행정동 유동인구 상위 10개')
    plt.xlabel('도착 행정동 코드')
    plt.ylabel('이동인구 합계')
    plt.show()

    요일순서 = ['월', '화', '수', '목', '금', '토', '일']

    # 도착 행정동 코드와 성별 나이 요일을 그룹화하고, 각 그룹의 이동인구(합)을 합산
    grouped_df = filtered_df_D.groupby(['도착 행정동 코드', '성별'])['이동인구(합)'].sum().reset_index()
    grouped_df_2 = filtered_df_D.groupby(['도착 행정동 코드', '나이'])['이동인구(합)'].sum().reset_index()
    grouped_df_3 = filtered_df_D.groupby(['도착 행정동 코드', '요일'])['이동인구(합)'].sum().reset_index()
    grouped_df_age_gender = filtered_df_D.groupby(['도착 행정동 코드', '나이', '성별'])['이동인구(합)'].sum().unstack().reset_index()


    # 상위 10개 도착 시군 구 코드만 선택
    top_regions = sorted_df.index.tolist()
    # grouped_df = grouped_df[grouped_df['도착 행정동 코드'].isin(top_regions)]
    # grouped_df_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'].isin(top_regions)]
    grouped_df_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'].isin(top_regions)]

    for region in top_regions:
        df_region = grouped_df[grouped_df['도착 행정동 코드'] == region]
        df_region_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'] == region]
        df_region_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'] == region]
        
        # 요일순서에 따라 데이터 정렬
        df_region_3['요일'] = pd.Categorical(df_region_3['요일'], categories=요일순서, ordered=True)
        df_region_3.sort_values(by='요일', inplace=True)
        
        # 서브 플롯 설정 (1x3)
        plt.figure(figsize=(18,6))

        # 원형그래프
        plt.subplot(1, 3, 1)
        plt.pie(df_region['이동인구(합)'], labels=df_region['성별'], autopct='%1.1f%%')
        plt.title(f'{region} - 성별 분포 ')
        
        # # 꺽은선그래프
        # plt.subplot(1, 3, 2)
        # df_region_2.set_index('나이')['이동인구(합)'].plot(kind='line', marker='o')
        # plt.title(f'{region} - 나이 ')
        
        # 막대그래프
        plt.subplot(1 , 3 , 2)
        df_region_3.set_index('요일')['이동인구(합)'].plot(kind='bar', color='skyblue')
        plt.title(f'{region} - 요일 ')

        # 꺽은선 그래프 & 막대그래프로 나이-성별재분류 추가 
        plt.subplot(1, 3, 2)
        df_region_age_gender = grouped_df_age_gender[grouped_df_age_gender['도착 행정동 코드'] == region].set_index('나이')
        # df_region_age_gender.plot(kind='bar', stacked=True, figsize=(6, 4))  # 사이즈 조정
        df_region_age_gender.plot(figsize=(6, 4))   # 사이즈 조정
        plt.title(f'Age and Gender Distribution in {region}')
        plt.xlabel('Age')
        plt.ylabel('Count')
        plt.grid()
        
        # 그래프 출력
        plt.show()


## 송파구에 가장 많이 방문하는 행정동의 성별, 나이 및 세대 별 성별 비교

In [36]:
    # 강남구는 도착 시군구 코드가 '강남구'
    gangnam_code = "송파구"

    # 강남구 데이터 필터링
    df_gangnam_G = df_2022_Q1[df_2022_Q1['도착 시군구 코드'] == gangnam_code]

    # 도착 행정동 별로 그룹화하여 이동인구(합) 합계 계산
    df_gangnam_G_grouped = df_gangnam_G.groupby('도착 행정동 코드')['이동인구(합)'].sum().nlargest(10)
    #df_gangnam_G_grouped = df_gangnam_G.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    # pandas DataFrame으로 변환
    df_gangnam_G_grouped = df_gangnam_G_grouped.compute()

    # 원 그래프
    plt.subplot(1,3,2)
    plt.pie(df_gangnam_G_grouped.values,
            labels=df_gangnam_G_grouped.index,
            autopct='%1.1f%%')
    plt.title("송파구 기준 가장 많이 방문하는 행정동 상위 10개")

    # 레이아웃 조절 및 차트 출력
    plt.tight_layout()
    plt.show()

    filtered_df_D = df_2022_Q1[(df_2022_Q1['도착 시군구 코드'] == '송파구')]

    filtered_df_D = filtered_df_D.compute()

    sorted_df = filtered_df_D.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    plt.figure(figsize=(10,6))
    sorted_df.plot(kind='bar', color='skyblue')
    plt.title('송파구 내의 행정동 유동인구 상위 10개')
    plt.xlabel('도착 행정동 코드')
    plt.ylabel('이동인구 합계')
    plt.show()

    요일순서 = ['월', '화', '수', '목', '금', '토', '일']

    # 도착 행정동 코드와 성별 나이 요일을 그룹화하고, 각 그룹의 이동인구(합)을 합산
    grouped_df = filtered_df_D.groupby(['도착 행정동 코드', '성별'])['이동인구(합)'].sum().reset_index()
    grouped_df_2 = filtered_df_D.groupby(['도착 행정동 코드', '나이'])['이동인구(합)'].sum().reset_index()
    grouped_df_3 = filtered_df_D.groupby(['도착 행정동 코드', '요일'])['이동인구(합)'].sum().reset_index()
    grouped_df_age_gender = filtered_df_D.groupby(['도착 행정동 코드', '나이', '성별'])['이동인구(합)'].sum().unstack().reset_index()


    # 상위 10개 도착 시군 구 코드만 선택
    top_regions = sorted_df.index.tolist()
    # grouped_df = grouped_df[grouped_df['도착 행정동 코드'].isin(top_regions)]
    # grouped_df_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'].isin(top_regions)]
    grouped_df_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'].isin(top_regions)]

    for region in top_regions:
        df_region = grouped_df[grouped_df['도착 행정동 코드'] == region]
        df_region_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'] == region]
        df_region_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'] == region]
        
        # 요일순서에 따라 데이터 정렬
        df_region_3['요일'] = pd.Categorical(df_region_3['요일'], categories=요일순서, ordered=True)
        df_region_3.sort_values(by='요일', inplace=True)
        
        # 서브 플롯 설정 (1x3)
        plt.figure(figsize=(18,6))

        # 원형그래프
        plt.subplot(1, 3, 1)
        plt.pie(df_region['이동인구(합)'], labels=df_region['성별'], autopct='%1.1f%%')
        plt.title(f'{region} - 성별 분포 ')
        
        # # 꺽은선그래프
        # plt.subplot(1, 3, 2)
        # df_region_2.set_index('나이')['이동인구(합)'].plot(kind='line', marker='o')
        # plt.title(f'{region} - 나이 ')
        
        # 막대그래프
        plt.subplot(1 , 3 , 2)
        df_region_3.set_index('요일')['이동인구(합)'].plot(kind='bar', color='skyblue')
        plt.title(f'{region} - 요일 ')

        # 꺽은선 그래프 & 막대그래프로 나이-성별재분류 추가 
        plt.subplot(1, 3, 2)
        df_region_age_gender = grouped_df_age_gender[grouped_df_age_gender['도착 행정동 코드'] == region].set_index('나이')
        # df_region_age_gender.plot(kind='bar', stacked=True, figsize=(6, 4))  # 사이즈 조정
        df_region_age_gender.plot(figsize=(6, 4))   # 사이즈 조정
        plt.title(f'Age and Gender Distribution in {region}')
        plt.xlabel('Age')
        plt.ylabel('Count')
        plt.grid()
        
        # 그래프 출력
        plt.show()


## 서초구에 가장 많이 방문하는 행정동의 성별, 나이 및 세대 별 성별 비교

In [37]:
    # 강남구는 도착 시군구 코드가 '강남구'
    gangnam_code = "서초구"

    # 강남구 데이터 필터링
    df_gangnam_G = df_2022_Q1[df_2022_Q1['도착 시군구 코드'] == gangnam_code]

    # 도착 행정동 별로 그룹화하여 이동인구(합) 합계 계산
    df_gangnam_G_grouped = df_gangnam_G.groupby('도착 행정동 코드')['이동인구(합)'].sum().nlargest(10)
    #df_gangnam_G_grouped = df_gangnam_G.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    # pandas DataFrame으로 변환
    df_gangnam_G_grouped = df_gangnam_G_grouped.compute()

    # 원 그래프
    plt.subplot(1,3,2)
    plt.pie(df_gangnam_G_grouped.values,
            labels=df_gangnam_G_grouped.index,
            autopct='%1.1f%%')
    plt.title("서초구 기준 가장 많이 방문하는 행정동 상위 10개")

    # 레이아웃 조절 및 차트 출력
    plt.tight_layout()
    plt.show()

    filtered_df_D = df_2022_Q1[(df_2022_Q1['도착 시군구 코드'] == '서초구')]

    filtered_df_D = filtered_df_D.compute()

    sorted_df = filtered_df_D.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    plt.figure(figsize=(10,6))
    sorted_df.plot(kind='bar', color='skyblue')
    plt.title('서초구 내의 행정동 유동인구 상위 10개')
    plt.xlabel('도착 행정동 코드')
    plt.ylabel('이동인구 합계')
    plt.show()

    요일순서 = ['월', '화', '수', '목', '금', '토', '일']

    # 도착 행정동 코드와 성별 나이 요일을 그룹화하고, 각 그룹의 이동인구(합)을 합산
    grouped_df = filtered_df_D.groupby(['도착 행정동 코드', '성별'])['이동인구(합)'].sum().reset_index()
    grouped_df_2 = filtered_df_D.groupby(['도착 행정동 코드', '나이'])['이동인구(합)'].sum().reset_index()
    grouped_df_3 = filtered_df_D.groupby(['도착 행정동 코드', '요일'])['이동인구(합)'].sum().reset_index()
    grouped_df_age_gender = filtered_df_D.groupby(['도착 행정동 코드', '나이', '성별'])['이동인구(합)'].sum().unstack().reset_index()


    # 상위 10개 도착 시군 구 코드만 선택
    top_regions = sorted_df.index.tolist()
    # grouped_df = grouped_df[grouped_df['도착 행정동 코드'].isin(top_regions)]
    # grouped_df_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'].isin(top_regions)]
    grouped_df_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'].isin(top_regions)]

    for region in top_regions:
        df_region = grouped_df[grouped_df['도착 행정동 코드'] == region]
        df_region_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'] == region]
        df_region_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'] == region]
        
        # 요일순서에 따라 데이터 정렬
        df_region_3['요일'] = pd.Categorical(df_region_3['요일'], categories=요일순서, ordered=True)
        df_region_3.sort_values(by='요일', inplace=True)
        
        # 서브 플롯 설정 (1x3)
        plt.figure(figsize=(18,6))

        # 원형그래프
        plt.subplot(1, 3, 1)
        plt.pie(df_region['이동인구(합)'], labels=df_region['성별'], autopct='%1.1f%%')
        plt.title(f'{region} - 성별 분포 ')
        
        # # 꺽은선그래프
        # plt.subplot(1, 3, 2)
        # df_region_2.set_index('나이')['이동인구(합)'].plot(kind='line', marker='o')
        # plt.title(f'{region} - 나이 ')
        
        # 막대그래프
        plt.subplot(1 , 3 , 2)
        df_region_3.set_index('요일')['이동인구(합)'].plot(kind='bar', color='skyblue')
        plt.title(f'{region} - 요일 ')

        # 꺽은선 그래프 & 막대그래프로 나이-성별재분류 추가 
        plt.subplot(1, 3, 2)
        df_region_age_gender = grouped_df_age_gender[grouped_df_age_gender['도착 행정동 코드'] == region].set_index('나이')
        # df_region_age_gender.plot(kind='bar', stacked=True, figsize=(6, 4))  # 사이즈 조정
        df_region_age_gender.plot(figsize=(6, 4))   # 사이즈 조정
        plt.title(f'Age and Gender Distribution in {region}')
        plt.xlabel('Age')
        plt.ylabel('Count')
        plt.grid()
        
        # 그래프 출력
        plt.show()


## 영등포구에 가장 많이 방문하는 행정동의 성별, 나이 및 세대 별 성별 비교

In [38]:
    # 강남구는 도착 시군구 코드가 '강남구'
    gangnam_code = "영등포구"

    # 강남구 데이터 필터링
    df_gangnam_G = df_2022_Q1[df_2022_Q1['도착 시군구 코드'] == gangnam_code]

    # 도착 행정동 별로 그룹화하여 이동인구(합) 합계 계산
    df_gangnam_G_grouped = df_gangnam_G.groupby('도착 행정동 코드')['이동인구(합)'].sum().nlargest(10)
    #df_gangnam_G_grouped = df_gangnam_G.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    # pandas DataFrame으로 변환
    df_gangnam_G_grouped = df_gangnam_G_grouped.compute()

    # 원 그래프
    plt.subplot(1,3,2)
    plt.pie(df_gangnam_G_grouped.values,
            labels=df_gangnam_G_grouped.index,
            autopct='%1.1f%%')
    plt.title("영등포구 기준 가장 많이 방문하는 행정동 상위 10개")

    # 레이아웃 조절 및 차트 출력
    plt.tight_layout()
    plt.show()

    filtered_df_D = df_2022_Q1[(df_2022_Q1['도착 시군구 코드'] == '영등포구')]

    filtered_df_D = filtered_df_D.compute()

    sorted_df = filtered_df_D.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    plt.figure(figsize=(10,6))
    sorted_df.plot(kind='bar', color='skyblue')
    plt.title('영등포구 내의 행정동 유동인구 상위 10개')
    plt.xlabel('도착 행정동 코드')
    plt.ylabel('이동인구 합계')
    plt.show()

    요일순서 = ['월', '화', '수', '목', '금', '토', '일']

    # 도착 행정동 코드와 성별 나이 요일을 그룹화하고, 각 그룹의 이동인구(합)을 합산
    grouped_df = filtered_df_D.groupby(['도착 행정동 코드', '성별'])['이동인구(합)'].sum().reset_index()
    grouped_df_2 = filtered_df_D.groupby(['도착 행정동 코드', '나이'])['이동인구(합)'].sum().reset_index()
    grouped_df_3 = filtered_df_D.groupby(['도착 행정동 코드', '요일'])['이동인구(합)'].sum().reset_index()
    grouped_df_age_gender = filtered_df_D.groupby(['도착 행정동 코드', '나이', '성별'])['이동인구(합)'].sum().unstack().reset_index()


    # 상위 10개 도착 시군 구 코드만 선택
    top_regions = sorted_df.index.tolist()
    # grouped_df = grouped_df[grouped_df['도착 행정동 코드'].isin(top_regions)]
    # grouped_df_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'].isin(top_regions)]
    grouped_df_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'].isin(top_regions)]

    for region in top_regions:
        df_region = grouped_df[grouped_df['도착 행정동 코드'] == region]
        df_region_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'] == region]
        df_region_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'] == region]
        
        # 요일순서에 따라 데이터 정렬
        df_region_3['요일'] = pd.Categorical(df_region_3['요일'], categories=요일순서, ordered=True)
        df_region_3.sort_values(by='요일', inplace=True)
        
        # 서브 플롯 설정 (1x3)
        plt.figure(figsize=(18,6))

        # 원형그래프
        plt.subplot(1, 3, 1)
        plt.pie(df_region['이동인구(합)'], labels=df_region['성별'], autopct='%1.1f%%')
        plt.title(f'{region} - 성별 분포 ')
        
        # # 꺽은선그래프
        # plt.subplot(1, 3, 2)
        # df_region_2.set_index('나이')['이동인구(합)'].plot(kind='line', marker='o')
        # plt.title(f'{region} - 나이 ')
        
        # 막대그래프
        plt.subplot(1 , 3 , 2)
        df_region_3.set_index('요일')['이동인구(합)'].plot(kind='bar', color='skyblue')
        plt.title(f'{region} - 요일 ')

        # 꺽은선 그래프 & 막대그래프로 나이-성별재분류 추가 
        plt.subplot(1, 3, 2)
        df_region_age_gender = grouped_df_age_gender[grouped_df_age_gender['도착 행정동 코드'] == region].set_index('나이')
        # df_region_age_gender.plot(kind='bar', stacked=True, figsize=(6, 4))  # 사이즈 조정
        df_region_age_gender.plot(figsize=(6, 4))   # 사이즈 조정
        plt.title(f'Age and Gender Distribution in {region}')
        plt.xlabel('Age')
        plt.ylabel('Count')
        plt.grid()
        
        # 그래프 출력
        plt.show()


## 용산구에 가장 많이 방문하는 행정동의 성별, 나이 및 세대 별 성별 비교

In [39]:
    # 강남구는 도착 시군구 코드가 '강남구'
    gangnam_code = "용산구"

    # 강남구 데이터 필터링
    df_gangnam_G = df_2022_Q1[df_2022_Q1['도착 시군구 코드'] == gangnam_code]

    # 도착 행정동 별로 그룹화하여 이동인구(합) 합계 계산
    df_gangnam_G_grouped = df_gangnam_G.groupby('도착 행정동 코드')['이동인구(합)'].sum().nlargest(10)
    #df_gangnam_G_grouped = df_gangnam_G.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    # pandas DataFrame으로 변환
    df_gangnam_G_grouped = df_gangnam_G_grouped.compute()

    # 원 그래프
    plt.subplot(1,3,2)
    plt.pie(df_gangnam_G_grouped.values,
            labels=df_gangnam_G_grouped.index,
            autopct='%1.1f%%')
    plt.title("용산구 기준 가장 많이 방문하는 행정동 상위 10개")

    # 레이아웃 조절 및 차트 출력
    plt.tight_layout()
    plt.show()

    filtered_df_D = df_2022_Q1[(df_2022_Q1['도착 시군구 코드'] == '용산구')]

    filtered_df_D = filtered_df_D.compute()

    sorted_df = filtered_df_D.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    plt.figure(figsize=(10,6))
    sorted_df.plot(kind='bar', color='skyblue')
    plt.title('용산구 내의 행정동 유동인구 상위 10개')
    plt.xlabel('도착 행정동 코드')
    plt.ylabel('이동인구 합계')
    plt.show()

    요일순서 = ['월', '화', '수', '목', '금', '토', '일']

    # 도착 행정동 코드와 성별 나이 요일을 그룹화하고, 각 그룹의 이동인구(합)을 합산
    grouped_df = filtered_df_D.groupby(['도착 행정동 코드', '성별'])['이동인구(합)'].sum().reset_index()
    grouped_df_2 = filtered_df_D.groupby(['도착 행정동 코드', '나이'])['이동인구(합)'].sum().reset_index()
    grouped_df_3 = filtered_df_D.groupby(['도착 행정동 코드', '요일'])['이동인구(합)'].sum().reset_index()
    grouped_df_age_gender = filtered_df_D.groupby(['도착 행정동 코드', '나이', '성별'])['이동인구(합)'].sum().unstack().reset_index()


    # 상위 10개 도착 시군 구 코드만 선택
    top_regions = sorted_df.index.tolist()
    # grouped_df = grouped_df[grouped_df['도착 행정동 코드'].isin(top_regions)]
    # grouped_df_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'].isin(top_regions)]
    grouped_df_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'].isin(top_regions)]

    for region in top_regions:
        df_region = grouped_df[grouped_df['도착 행정동 코드'] == region]
        df_region_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'] == region]
        df_region_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'] == region]
        
        # 요일순서에 따라 데이터 정렬
        df_region_3['요일'] = pd.Categorical(df_region_3['요일'], categories=요일순서, ordered=True)
        df_region_3.sort_values(by='요일', inplace=True)
        
        # 서브 플롯 설정 (1x3)
        plt.figure(figsize=(18,6))

        # 원형그래프
        plt.subplot(1, 3, 1)
        plt.pie(df_region['이동인구(합)'], labels=df_region['성별'], autopct='%1.1f%%')
        plt.title(f'{region} - 성별 분포 ')
        
        # # 꺽은선그래프
        # plt.subplot(1, 3, 2)
        # df_region_2.set_index('나이')['이동인구(합)'].plot(kind='line', marker='o')
        # plt.title(f'{region} - 나이 ')
        
        # 막대그래프
        plt.subplot(1 , 3 , 2)
        df_region_3.set_index('요일')['이동인구(합)'].plot(kind='bar', color='skyblue')
        plt.title(f'{region} - 요일 ')

        # 꺽은선 그래프 & 막대그래프로 나이-성별재분류 추가 
        plt.subplot(1, 3, 2)
        df_region_age_gender = grouped_df_age_gender[grouped_df_age_gender['도착 행정동 코드'] == region].set_index('나이')
        # df_region_age_gender.plot(kind='bar', stacked=True, figsize=(6, 4))  # 사이즈 조정
        df_region_age_gender.plot(figsize=(6, 4))   # 사이즈 조정
        plt.title(f'Age and Gender Distribution in {region}')
        plt.xlabel('Age')
        plt.ylabel('Count')
        plt.grid()
        
        # 그래프 출력
        plt.show()


## 중구에 가장 많이 방문하는 행정동의 성별, 나이 및 세대 별 성별 비교

In [40]:
    # 강남구는 도착 시군구 코드가 '강남구'
    gangnam_code = "중구"

    # 강남구 데이터 필터링
    df_gangnam_G = df_2022_Q1[df_2022_Q1['도착 시군구 코드'] == gangnam_code]

    # 도착 행정동 별로 그룹화하여 이동인구(합) 합계 계산
    df_gangnam_G_grouped = df_gangnam_G.groupby('도착 행정동 코드')['이동인구(합)'].sum().nlargest(10)
    #df_gangnam_G_grouped = df_gangnam_G.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    # pandas DataFrame으로 변환
    df_gangnam_G_grouped = df_gangnam_G_grouped.compute()

    # 원 그래프
    plt.subplot(1,3,2)
    plt.pie(df_gangnam_G_grouped.values,
            labels=df_gangnam_G_grouped.index,
            autopct='%1.1f%%')
    plt.title("중구 기준 가장 많이 방문하는 행정동 상위 10개")

    # 레이아웃 조절 및 차트 출력
    plt.tight_layout()
    plt.show()

    filtered_df_D = df_2022_Q1[(df_2022_Q1['도착 시군구 코드'] == '중구')]

    filtered_df_D = filtered_df_D.compute()

    sorted_df = filtered_df_D.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    plt.figure(figsize=(10,6))
    sorted_df.plot(kind='bar', color='skyblue')
    plt.title('중구 내의 행정동 유동인구 상위 10개')
    plt.xlabel('도착 행정동 코드')
    plt.ylabel('이동인구 합계')
    plt.show()

    요일순서 = ['월', '화', '수', '목', '금', '토', '일']

    # 도착 행정동 코드와 성별 나이 요일을 그룹화하고, 각 그룹의 이동인구(합)을 합산
    grouped_df = filtered_df_D.groupby(['도착 행정동 코드', '성별'])['이동인구(합)'].sum().reset_index()
    grouped_df_2 = filtered_df_D.groupby(['도착 행정동 코드', '나이'])['이동인구(합)'].sum().reset_index()
    grouped_df_3 = filtered_df_D.groupby(['도착 행정동 코드', '요일'])['이동인구(합)'].sum().reset_index()
    grouped_df_age_gender = filtered_df_D.groupby(['도착 행정동 코드', '나이', '성별'])['이동인구(합)'].sum().unstack().reset_index()


    # 상위 10개 도착 시군 구 코드만 선택
    top_regions = sorted_df.index.tolist()
    # grouped_df = grouped_df[grouped_df['도착 행정동 코드'].isin(top_regions)]
    # grouped_df_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'].isin(top_regions)]
    grouped_df_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'].isin(top_regions)]

    for region in top_regions:
        df_region = grouped_df[grouped_df['도착 행정동 코드'] == region]
        df_region_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'] == region]
        df_region_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'] == region]
        
        # 요일순서에 따라 데이터 정렬
        df_region_3['요일'] = pd.Categorical(df_region_3['요일'], categories=요일순서, ordered=True)
        df_region_3.sort_values(by='요일', inplace=True)
        
        # 서브 플롯 설정 (1x3)
        plt.figure(figsize=(18,6))

        # 원형그래프
        plt.subplot(1, 3, 1)
        plt.pie(df_region['이동인구(합)'], labels=df_region['성별'], autopct='%1.1f%%')
        plt.title(f'{region} - 성별 분포 ')
        
        # # 꺽은선그래프
        # plt.subplot(1, 3, 2)
        # df_region_2.set_index('나이')['이동인구(합)'].plot(kind='line', marker='o')
        # plt.title(f'{region} - 나이 ')
        
        # 막대그래프
        plt.subplot(1 , 3 , 2)
        df_region_3.set_index('요일')['이동인구(합)'].plot(kind='bar', color='skyblue')
        plt.title(f'{region} - 요일 ')

        # 꺽은선 그래프 & 막대그래프로 나이-성별재분류 추가 
        plt.subplot(1, 3, 2)
        df_region_age_gender = grouped_df_age_gender[grouped_df_age_gender['도착 행정동 코드'] == region].set_index('나이')
        # df_region_age_gender.plot(kind='bar', stacked=True, figsize=(6, 4))  # 사이즈 조정
        df_region_age_gender.plot(figsize=(6, 4))   # 사이즈 조정
        plt.title(f'Age and Gender Distribution in {region}')
        plt.xlabel('Age')
        plt.ylabel('Count')
        plt.grid()
        
        # 그래프 출력
        plt.show()


## 종로구에 가장 많이 방문하는 행정동의 성별, 나이 및 세대 별 성별 비교

In [41]:
    # 강남구는 도착 시군구 코드가 '강남구'
    gangnam_code = "종로구"

    # 강남구 데이터 필터링
    df_gangnam_G = df_2022_Q1[df_2022_Q1['도착 시군구 코드'] == gangnam_code]

    # 도착 행정동 별로 그룹화하여 이동인구(합) 합계 계산
    df_gangnam_G_grouped = df_gangnam_G.groupby('도착 행정동 코드')['이동인구(합)'].sum().nlargest(10)
    #df_gangnam_G_grouped = df_gangnam_G.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    # pandas DataFrame으로 변환
    df_gangnam_G_grouped = df_gangnam_G_grouped.compute()

    # 원 그래프
    plt.subplot(1,3,2)
    plt.pie(df_gangnam_G_grouped.values,
            labels=df_gangnam_G_grouped.index,
            autopct='%1.1f%%')
    plt.title("종로구 기준 가장 많이 방문하는 행정동 상위 10개")

    # 레이아웃 조절 및 차트 출력
    plt.tight_layout()
    plt.show()

    filtered_df_D = df_2022_Q1[(df_2022_Q1['도착 시군구 코드'] == '종로구')]

    filtered_df_D = filtered_df_D.compute()

    sorted_df = filtered_df_D.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    plt.figure(figsize=(10,6))
    sorted_df.plot(kind='bar', color='skyblue')
    plt.title('종로구 내의 행정동 유동인구 상위 10개')
    plt.xlabel('도착 행정동 코드')
    plt.ylabel('이동인구 합계')
    plt.show()

    요일순서 = ['월', '화', '수', '목', '금', '토', '일']

    # 도착 행정동 코드와 성별 나이 요일을 그룹화하고, 각 그룹의 이동인구(합)을 합산
    grouped_df = filtered_df_D.groupby(['도착 행정동 코드', '성별'])['이동인구(합)'].sum().reset_index()
    grouped_df_2 = filtered_df_D.groupby(['도착 행정동 코드', '나이'])['이동인구(합)'].sum().reset_index()
    grouped_df_3 = filtered_df_D.groupby(['도착 행정동 코드', '요일'])['이동인구(합)'].sum().reset_index()
    grouped_df_age_gender = filtered_df_D.groupby(['도착 행정동 코드', '나이', '성별'])['이동인구(합)'].sum().unstack().reset_index()


    # 상위 10개 도착 시군 구 코드만 선택
    top_regions = sorted_df.index.tolist()
    # grouped_df = grouped_df[grouped_df['도착 행정동 코드'].isin(top_regions)]
    # grouped_df_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'].isin(top_regions)]
    grouped_df_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'].isin(top_regions)]

    for region in top_regions:
        df_region = grouped_df[grouped_df['도착 행정동 코드'] == region]
        df_region_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'] == region]
        df_region_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'] == region]
        
        # 요일순서에 따라 데이터 정렬
        df_region_3['요일'] = pd.Categorical(df_region_3['요일'], categories=요일순서, ordered=True)
        df_region_3.sort_values(by='요일', inplace=True)
        
        # 서브 플롯 설정 (1x3)
        plt.figure(figsize=(18,6))

        # 원형그래프
        plt.subplot(1, 3, 1)
        plt.pie(df_region['이동인구(합)'], labels=df_region['성별'], autopct='%1.1f%%')
        plt.title(f'{region} - 성별 분포 ')
        
        # # 꺽은선그래프
        # plt.subplot(1, 3, 2)
        # df_region_2.set_index('나이')['이동인구(합)'].plot(kind='line', marker='o')
        # plt.title(f'{region} - 나이 ')
        
        # 막대그래프
        plt.subplot(1 , 3 , 2)
        df_region_3.set_index('요일')['이동인구(합)'].plot(kind='bar', color='skyblue')
        plt.title(f'{region} - 요일 ')

        # 꺽은선 그래프 & 막대그래프로 나이-성별재분류 추가 
        plt.subplot(1, 3, 2)
        df_region_age_gender = grouped_df_age_gender[grouped_df_age_gender['도착 행정동 코드'] == region].set_index('나이')
        # df_region_age_gender.plot(kind='bar', stacked=True, figsize=(6, 4))  # 사이즈 조정
        df_region_age_gender.plot(figsize=(6, 4))   # 사이즈 조정
        plt.title(f'Age and Gender Distribution in {region}')
        plt.xlabel('Age')
        plt.ylabel('Count')
        plt.grid()
        
        # 그래프 출력
        plt.show()


## 성동구에 가장 많이 방문하는 행정동의 성별, 나이 및 세대 별 성별 비교

In [42]:
    # 강남구는 도착 시군구 코드가 '강남구'
    gangnam_code = "성동구"

    # 강남구 데이터 필터링
    df_gangnam_G = df_2022_Q1[df_2022_Q1['도착 시군구 코드'] == gangnam_code]

    # 도착 행정동 별로 그룹화하여 이동인구(합) 합계 계산
    df_gangnam_G_grouped = df_gangnam_G.groupby('도착 행정동 코드')['이동인구(합)'].sum().nlargest(10)
    #df_gangnam_G_grouped = df_gangnam_G.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    # pandas DataFrame으로 변환
    df_gangnam_G_grouped = df_gangnam_G_grouped.compute()

    # 원 그래프
    plt.subplot(1,3,2)
    plt.pie(df_gangnam_G_grouped.values,
            labels=df_gangnam_G_grouped.index,
            autopct='%1.1f%%')
    plt.title("성동구 기준 가장 많이 방문하는 행정동 상위 10개")

    # 레이아웃 조절 및 차트 출력
    plt.tight_layout()
    plt.show()

    filtered_df_D = df_2022_Q1[(df_2022_Q1['도착 시군구 코드'] == '성동구')]

    filtered_df_D = filtered_df_D.compute()

    sorted_df = filtered_df_D.groupby('도착 행정동 코드')['이동인구(합)'].sum().sort_values(ascending=False).head(10)

    plt.figure(figsize=(10,6))
    sorted_df.plot(kind='bar', color='skyblue')
    plt.title('성동구 내의 행정동 유동인구 상위 10개')
    plt.xlabel('도착 행정동 코드')
    plt.ylabel('이동인구 합계')
    plt.show()

    요일순서 = ['월', '화', '수', '목', '금', '토', '일']

    # 도착 행정동 코드와 성별 나이 요일을 그룹화하고, 각 그룹의 이동인구(합)을 합산
    grouped_df = filtered_df_D.groupby(['도착 행정동 코드', '성별'])['이동인구(합)'].sum().reset_index()
    grouped_df_2 = filtered_df_D.groupby(['도착 행정동 코드', '나이'])['이동인구(합)'].sum().reset_index()
    grouped_df_3 = filtered_df_D.groupby(['도착 행정동 코드', '요일'])['이동인구(합)'].sum().reset_index()
    grouped_df_age_gender = filtered_df_D.groupby(['도착 행정동 코드', '나이', '성별'])['이동인구(합)'].sum().unstack().reset_index()


    # 상위 10개 도착 시군 구 코드만 선택
    top_regions = sorted_df.index.tolist()
    # grouped_df = grouped_df[grouped_df['도착 행정동 코드'].isin(top_regions)]
    # grouped_df_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'].isin(top_regions)]
    grouped_df_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'].isin(top_regions)]

    for region in top_regions:
        df_region = grouped_df[grouped_df['도착 행정동 코드'] == region]
        df_region_2 = grouped_df_2[grouped_df_2['도착 행정동 코드'] == region]
        df_region_3 = grouped_df_3[grouped_df_3['도착 행정동 코드'] == region]
        
        # 요일순서에 따라 데이터 정렬
        df_region_3['요일'] = pd.Categorical(df_region_3['요일'], categories=요일순서, ordered=True)
        df_region_3.sort_values(by='요일', inplace=True)
        
        # 서브 플롯 설정 (1x3)
        plt.figure(figsize=(18,6))

        # 원형그래프
        plt.subplot(1, 3, 1)
        plt.pie(df_region['이동인구(합)'], labels=df_region['성별'], autopct='%1.1f%%')
        plt.title(f'{region} - 성별 분포 ')
        
        # # 꺽은선그래프
        # plt.subplot(1, 3, 2)
        # df_region_2.set_index('나이')['이동인구(합)'].plot(kind='line', marker='o')
        # plt.title(f'{region} - 나이 ')
        
        # 막대그래프
        plt.subplot(1 , 3 , 2)
        df_region_3.set_index('요일')['이동인구(합)'].plot(kind='bar', color='skyblue')
        plt.title(f'{region} - 요일 ')

        # 꺽은선 그래프 & 막대그래프로 나이-성별재분류 추가 
        plt.subplot(1, 3, 2)
        df_region_age_gender = grouped_df_age_gender[grouped_df_age_gender['도착 행정동 코드'] == region].set_index('나이')
        # df_region_age_gender.plot(kind='bar', stacked=True, figsize=(6, 4))  # 사이즈 조정
        df_region_age_gender.plot(figsize=(6, 4))   # 사이즈 조정
        plt.title(f'Age and Gender Distribution in {region}')
        plt.xlabel('Age')
        plt.ylabel('Count')
        plt.grid()
        
        # 그래프 출력
        plt.show()


# 강남구 데이터만 필터링
Gangnamo_df = filtered_df_AE[filtered_df_AE['도착 시군구 코드'] == '강남구']


seocho_df['이동인구(합)'] = pd.to_numeric(seocho_df['이동인구(합)'], errors='coerce')


grouped_seocho = seocho_df.groupby('도착 행정동 코드')['이동인구(합)'].sum()

ranked_seocho = grouped_seocho.sort_values(ascending=False)

print(ranked_seocho)


# 강남구 기준 상위 10 행정동 코드 
top_5 = ranked_seocho.head(5)

# 막대 그래프 생성
sns.barplot(x=top_5.index, y=top_5.values)

# 그래프 제목 및 레이블 설정
plt.title('Top 5 도착 행정동 코드 in 서초구')
plt.xlabel('도착 행정동 코드')
plt.ylabel('이동 인구수')

# 그래프 출력
plt.show()